In [1]:
"""IDMT SMT Bass Dataset Loader.
.. admonition:: Dataset Info
    :class: dropdown
    The IDMT-SMT-Bass database is a large database for automatic bass transcription and signal processing.
    The overall duration of the audio material is approx. 3.6 hours.
    The dataset consists of approx. 4300 WAV files (44.1 kHz, 24bit) with single recorded notes.
    Overall, 10 different bass-related playing techniques namely 5 plucking styles
      fingerstyle (FS) 
      picked (PK)
      muted (MU)
      slap-thumb (ST)
      slap-pluck (SP)
    and 5 expression styles
      normal (NO)
      vibrato (VI)
      bending (BE)
      harmonics (HA)
      dead-note (DN)
    are incorporated. A further explaination of the playing techniques is provided in the paper for the same.

 For each of the three expression techniques (vibrato, bending, slide), two subclasses were defined in [2]:
 fast vibrato, slow vibrato
 semi-tone bending, quarter-note bending
 slide up, slide down (recorded with fretless bass guitar)
 3 different 4-string electric bass guitars, each with 3 different pick-up settings were used for recording.

 The notes cover the common pitch range of a 4-string bass guitar from E1 (41.2 Hz) to G3 (196.0 Hz). 
 THE DATASET CAN BE DOWNLOADED BY ASKING FOR ACCESS FROM THE LINK PROVIDED IN THE REMOTES SECTION.
"""

'IDMT SMT Bass Dataset Loader.\n.. admonition:: Dataset Info\n    :class: dropdown\n    The IDMT-SMT-Bass database is a large database for automatic bass transcription and signal processing.\n    The overall duration of the audio material is approx. 3.6 hours.\n    The dataset consists of approx. 4300 WAV files (44.1 kHz, 24bit) with single recorded notes.\n    Overall, 10 different bass-related playing techniques namely 5 plucking styles\n      fingerstyle (FS) \n      picked (PK)\n      muted (MU)\n      slap-thumb (ST)\n      slap-pluck (SP)\n    and 5 expression styles\n      normal (NO)\n      vibrato (VI)\n      bending (BE)\n      harmonics (HA)\n      dead-note (DN)\n    are incorporated. A further explaination of the playing techniques is provided in the paper for the same.\n\n For each of the three expression techniques (vibrato, bending, slide), two subclasses were defined in [2]:\n fast vibrato, slow vibrato\n semi-tone bending, quarter-note bending\n slide up, slide down (

In [2]:
import csv
import os
from typing import BinaryIO, Optional, Tuple
import json

import librosa
import numpy as np

from mirdata import core, download_utils, io, jams_utils
from mirdata.validate import md5


In [3]:
BIBTEX = """@INPROCEEDINGS{5495945, 
  author={Abeßer, Jakob and Lukashevich, Hanna and Schuller, Gerald},
  booktitle={2010 IEEE International Conference on Acoustics, Speech and Signal Processing},
  title={Feature-based extraction of plucking and expression styles of the electric bass guitar}, 
  year={2010},  volume={}, 
  number={},
  pages={2290-2293},
  doi={10.1109/ICASSP.2010.5495945}
}"""

In [4]:
INDEXES = {
    "default": "1.1",
    "test": "1.1",
    "1.1": core.Index(filename="IDMT_SMT_Bass_index.json"),
}

In [5]:
REMOTES = {
    "audio": download_utils.RemoteFileMetadata(
        filename="IDMT-SMT-BASS.zip",
        url="https://www.idmt.fraunhofer.de/en/publications/datasets/published-datasets-terms.html",
        checksum="884d77260aaaa3df7557f77a6b3aef18",
        destination_dir="IDMT-SMT-BASS",
    ),
    
}
    

In [6]:
LICENSE_INFO = "Creative Commons Attribution-ShareAlike 4.0 International License ('by-sa')"

In [7]:
Type_of_Bass_Guitar= {1: "Fame Baphomet NTB 4 String", 2: "Warwick 4 String", 3: "Yamaha BB604 Jazz Bass 4 String"}

In [8]:
class Track(core.Track):
    """IDMT-SMT-Bass Track class
    Args:
        track_id (str): track id of the track
    Attributes:
        audio_path (str): path of the audio file
        instrument_model (str): name of the bass guitar used
        string_number (int): The string corresponds to a standard 4 string bass tuning.
        track_id (str): track id
    """      
    
    def __init__(
        self,
        track_id,
        data_home,
        dataset_name,
        index,
        metadata,
    ):
        super().__init__(
            track_id,
            data_home,
            dataset_name,
            index,
            metadata,
        )

        self.audio_path = self.get_path("audio")

    @property
    def instrument_model(self):
        return self._track_metadata.get("instrument_model")
    
    @property
    def string_number(self):
        return self._track_metadata.get("string_number")
    
    @property
    def audio(self) -> Optional[Tuple[np.ndarray, float]]:
        """The track's audio
        Returns:
            * np.ndarray - audio signal
            * float - sample rate
        """
        return load_audio(self.audio_path)
    
    def to_jams(self):
        """Get the track's data in jams format
        Returns:
            jams.JAMS: the track's data in jams format
        """
        return jams_utils.jams_converter(
            audio_path=self.audio_path, metadata=self._track_metadata
        )


In [9]:
# -- this decorator allows this function to take a string or an open bytes file as input
# -- and in either case converts it to an open file handle.
# -- It also checks if the file exists
# -- and, if None is passed, None will be returned 

@io.coerce_to_bytes_io
def load_audio(fhandle: BinaryIO) -> Tuple[np.ndarray, float]:
    """Load an IDMT SMT Bass audio file.
    Args:
        fhandle (str or file-like): File-like object or path to audio file
    Returns:
        * np.ndarray - the mono audio signal
        * float - The sample rate of the audio file
    """
    return librosa.load(fhandle, sr=44100, mono=True)

In [10]:
# -- use this decorator so the docs are complete
@core.docstring_inherit(core.Dataset)
class Dataset(core.Dataset):
    """
    The IDMT-SMT-Bass dataset
    """

    def __init__(self, data_home=None, version="default"):
        super().__init__(
            data_home,
            version,
            name="IDMT-SMT-Bass",
            track_class=Track,
            bibtex=BIBTEX,
            indexes=INDEXES,
            remotes=REMOTES,
            license_info=LICENSE_INFO,
        )
# -- if your dataset has a top-level metadata file, write a loader for it here
    # -- you do not have to include this function if there is no metadata 
    @core.cached_property
    def _metadata(self):
        dataset_path = os.path.join(
            self.data_home, "IDMT-SMT-Bass"
        )
        metadata_index = {}
        
        for folder in os.listdir(dataset_path):
                     
            if '.' not in folder:
                for subfolder in os.listdir(os.path.join(dataset_path, folder)):
                    for track in os.listdir(os.path.join(dataset_path, folder, subfolder)):
                        tr_split = track.split("_")
                        str_num = int(tr_split[-2])
                        instrument_model_int = int(tr_split[1])
                        if instrument_model_int==1:
                            instrument_model =Type_of_Bass_Guitar[1]
                        elif instrument_model_int==2:
                            instrument_model =Type_of_Bass_Guitar[2]
                        else:
                            instrument_model=Type_of_Bass_Guitar[3]
                            metadata_index[track.replace(".wav", "")] = {
                            "string_number": str_num,
                            "instrument_model":instrument_model}
                        
        return metadata_index
# -- Copy any loaders you wrote that should be part of the Dataset class
    # -- use this core.copy_docs decorator to copy the docs from the original
    # -- load_ function
    @core.copy_docs(load_audio)
    def load_audio(self, *args, **kwargs):
        return load_audio(*args, **kwargs)